In [ ]:
# images augumentation (main)

import cv2
import os
import numpy as np
import random

# Augmentation functions
def adjust_brightness(img, factor=0.2):
    return cv2.convertScaleAbs(img, alpha=1, beta=random.uniform(-factor * 255, factor * 255))

def add_gaussian_noise(img, mean=0, std=10):
    noise = np.random.normal(mean, std, img.shape).astype(np.int16)
    return np.clip(img + noise, 0, 255).astype(np.uint8)

def adjust_contrast(img, factor_range=(0.8, 1.2)):
    factor = random.uniform(*factor_range)
    mean = np.mean(img)
    return np.clip((img - mean) * factor + mean, 0, 255).astype(np.uint8)

def color_jitter(img, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv[..., 0] = np.clip(hsv[..., 0] + random.uniform(-hue, hue) * 255, 0, 255)
    hsv[..., 1] = np.clip(hsv[..., 1] * random.uniform(1 - saturation, 1 + saturation), 0, 255)
    hsv[..., 2] = np.clip(hsv[..., 2] * random.uniform(1 - brightness, 1 + brightness), 0, 255)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return cv2.convertScaleAbs(img, alpha=random.uniform(1 - contrast, 1 + contrast), beta=0)

# Paths
input_folder = './chess_images/board_and_fen'
output_folder = './augumented_train_images/board_and_fen_train_50images'

os.makedirs(output_folder, exist_ok=True)

for i in range(0, 101):  # Change here for folders
    input_folder_path = os.path.join(input_folder, str(i))
    output_folder_path = os.path.join(output_folder, str(i))

    if not os.path.exists(input_folder_path):
        print(f"Skipping folder {i} (does not exist)")
        continue

    os.makedirs(output_folder_path, exist_ok=True)

    images = sorted([f for f in os.listdir(input_folder_path) if f.endswith('.jpg') or f.endswith('.png')])

    if len(images) != 5:
        print(f"Folder {i} does not contain exactly 5 images — skipping.")
        continue

    # Process each of the 5 images
    for img_index, image_name in enumerate(images):
        original_image_path = os.path.join(input_folder_path, image_name)
        img = cv2.imread(original_image_path)

        # Save the original image directly
        cv2.imwrite(os.path.join(output_folder_path, image_name), img)

        # Generate 4 augmented versions per image
        for j in range(9): # Change here for how many you want
            augmented = img.copy()

            # Apply random augmentations (you can customize this part)
            if random.random() > 0.5:
                augmented = adjust_brightness(augmented)
            if random.random() > 0.5:
                augmented = add_gaussian_noise(augmented)
            if random.random() > 0.5:
                augmented = adjust_contrast(augmented)
            if random.random() > 0.5:
                augmented = color_jitter(augmented)

            augmented_image_path = os.path.join(output_folder_path, f"{image_name.split('.')[0]}_augmented_{j+1}.jpg")
            cv2.imwrite(augmented_image_path, augmented)

    # Copy the bw_board.fen file if it exists
    fen_file_path = os.path.join(input_folder_path, 'bw_board.fen')
    if os.path.exists(fen_file_path):
        output_fen_file_path = os.path.join(output_folder_path, 'bw_board.fen')
        with open(fen_file_path, 'r') as src, open(output_fen_file_path, 'w') as dst:
            dst.write(src.read())

print("Augmentation process complete.")
